# Download survey GeoTiffs

New, faster method for downloading DHS cluster images! Based on [this blog post by Noel Gorelick](https://gorelick.medium.com/fast-er-downloads-a2abd512aa26).

Thanks to Markus Pettersson, the original author of this code. 

Import, authenticate and initialize the earth-engine library

In [1]:
import ee

ee.Authenticate()

# Initialize the Google Earth Engine API with the high volume end-point.
# See https://developers.google.com/earth-engine/cloud/highvolume
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

Enter verification code: 4/1AbUR2VP3q13l1DBq2hxgmqM30-ZOUl9SYyw1GjdL_-iUYtmedI4leXSn9J0

Successfully saved authorization token.


In [2]:
# Import other libraries
import pandas as pd
import os
import satellite_sampling
import datetime

Read the csv file with survey points

In [3]:
data_dir = '/mimer/NOBACKUP/groups/globalpoverty1/cindy/eoml_ch_wb/data/'
dhs_cluster_file_path = os.path.join(data_dir, 'dhs_clusters.csv')
df = pd.read_csv(dhs_cluster_file_path)
df.head()

,country,survey_start_year,year,lat,lon,GID_1,GID_2,households,rural,iwi
0,angola,2006,2006,-12.169283,13.859255,AGO.2,AGO.2.9,20,0,68.096300
1,angola,2006,2006,-12.361700,14.778201,AGO.2,AGO.2.2,19,1,10.609053
2,angola,2006,2006,-10.827451,14.342114,AGO.7,AGO.7.1,25,1,29.973928
3,angola,2006,2006,-9.991311,14.898295,AGO.7,AGO.7.5,1,1,7.704135
4,angola,2006,2006,-11.518893,14.750437,AGO.7,AGO.7.10,24,1,13.371826


Split the dataframe into each country-year combination:

In [4]:
surveys_with_dfs = [(survey, survey_df.reset_index(drop=True)) for survey, survey_df in 
                    df.groupby(['country', 'year'])]

Function for checking if sample is already downloaded, in case the script needs to be restarted for some reason

In [5]:
def check_if_downloaded(row, save_dir, min_file_size=3145728):
    file_name = f'{row.name:05d}.tif'
    file_path = os.path.join(save_dir, file_name)
    
    # Check if file exists and is larger than min_file_size
    return os.path.isfile(file_path) and (os.stat(file_path).st_size > min_file_size)

Download each survey from Google Earth Engine

In [10]:
for survey, survey_df in surveys_with_dfs:
    country, year = survey
    print(f'Downloading images for {country}-{year}...'+
        datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))
       
    save_dir = os.path.join(data_dir, f'dhs_tifs/{country}_{year}')      
           
    # Check if survey is already fully/partially downloaded
    if os.path.exists(save_dir):
        is_downloaded = survey_df.apply(lambda row: check_if_downloaded(row, save_dir), axis=1)
        samples_to_download = survey_df[~is_downloaded]
    else:
        os.makedirs(save_dir)
        samples_to_download = survey_df
    
    # If there are no samples to download for this survey, continue to next one
    if len(samples_to_download) == 0:
        continue
    
    satellite_sampling.export_images(samples_to_download, save_dir, span_length=3)